# 第6回演習「正しいおやつの買い方を学習させる」

この演習ではニューラルネットワーク(単純パーセプトロン) の計算手順を習得します。<hr />

## 【例題】
ケメ子さんはロボットの「ブラッキィさん」に、遠足のおやつを買いに行かせようとしています。おやつは3種類しかなく、制約は「総額が500円を超えてはいけない」「1種類のおやつは1個まで」の2つだけです。簡単なルールですね。

<table border="1" style="margin-left: auto;margin-right: auto">
    <tr><th>おやつ1</th><th>おやつ2</th><th>おやつ3</th></tr>
    <tr><td>310円</td><td>220円</td><td>70円</td></tr>
</table> 
<br />
<img src="http://pfe.p.cyber-u.ac.jp/img/Python/Week6/robot.png" alt="ロボットのブラッキィさん" style="width:120px">

ブラッキィさんはあまり賢くなく、「総額500円を超えてはいけない」というルールを覚えられません。それだけでなく、個々のおやつの金額も覚えられません。このままでは「使えないロボット」の烙印を押され、廃棄されてしまいます!

しかし、その一方でブラッキィさんはニューラルネットワークによる学習機能を持っています。買ったおやつをケメ子さんに見せて

*   この買い方はNG!
*   この買い方はOK!

という指摘を何度も繰り返し受けると、しだいに適切な買い方を学習していきます。

In [1]:
class おやつの買い方(object):
    def __init__ (self,買い方の名前,購入有無のリスト,正解): # コンストラクタ
        self.買い方の名前=買い方の名前
        self.購入有無のリスト=購入有無のリスト # それぞれを「買うなら1、買わないなら0」
        self.正解=正解 # 「NGな買い方は1、OKは0」

ブラッキィさんの中には部品が4つ入っています。部品B、部品C、部品Dがそれぞれおやつ1、おやつ2、おやつ3の購入判断を担当します。そして、部品Aは「3種類のおやつ全体についての判断」を担当します。AはB、C、Dの3つを裏でまとめる司令塔のような役割です。

<img src="http://pfe.p.cyber-u.ac.jp/img/Python/Week6/snacks.png" alt="4つの部品" style="width:360px">

In [2]:
class 部品(object):
    def __init__ (self,部品名,中の値): # コンストラクタ
        self.部品名=部品名
        self.中の値=中の値
    def 増やす(self,増やす値):   # 指定された値を足す
        self.中の値=self.中の値+増やす値
    def 減らす(self,減らす値):   # 指定された値を引く
        self.中の値=self.中の値-減らす値

おやつ担当部品のリスト=[部品("部品B",1), 部品("部品C",3), 部品("部品D",8)] # 3つの部品BCDと中の値
部品A=部品("部品A",6)
おやつの個数=len(おやつ担当部品のリスト) # おやつの個数は、おやつ担当部品の数と同じ

おやつの組み合わせは8通りあり、その中の5通りを教師データとして用い、残りの3通りを正しく判断させることを目標とします。

In [3]:
教師データのリスト=[ # 5通りの教師データ
    おやつの買い方("買い方1",[1,1,1],"NG"),
    おやつの買い方("買い方2",[1,1,0],"NG"),
    おやつの買い方("買い方3",[1,0,1],"OK"),
    おやつの買い方("買い方4",[0,1,1],"OK"),
    おやつの買い方("買い方5",[1,0,0],"OK")]

未知データのリスト=[ # 3通りの未知データ
    おやつの買い方("買い方6",[0,1,0],"OK"),
    おやつの買い方("買い方7",[0,0,1],"OK"),
    おやつの買い方("買い方8",[0,0,0],"OK")]

買い方判定の関数と学習の関数は、以下の通りです。

In [4]:
def 買い方判定(買い方データ,部品A): # 部品B,C,D,Aに買い方の可否を判定させる
    合計=0
    for 順序 in range(おやつの個数):
        合計=合計+買い方データ.購入有無のリスト[順序]*おやつ担当部品のリスト[順序].中の値

    print(str(合計)+" > " + str(部品A.中の値)+" ", end="") # 比較の式を表示
    
    if 合計 > 部品A.中の値 : 
        return "NG" # 判定はNGで、部品Aが興奮する
    else:
        return "OK" # 判定はOK 

def 与えられたデータで学習(買い方データ,部品A):
    print(買い方データ.買い方の名前+"は ",end="")
    print(",".join([str(x) for x in 買い方データ.購入有無のリスト])+"、",end="")
    判定結果=買い方判定(買い方データ,部品A)
    print("の比較により判定は"+判定結果+"、",end="")

    # 判定結果と正解に基づき、BCDAの中の値を増減させる
    if 買い方データ.正解 == "OK" and 判定結果 == "NG": # 誤りタイプ1
        # BCDを1ずつ減らし、Aを1増やす
        for 順序 in range(おやつの個数): # 順序は 0,1,2
            if 買い方データ.購入有無のリスト[順序]==1: # この部品がおやつを買う、と言った時
                おやつ担当部品のリスト[順序].減らす(1)
        部品A.増やす(1)            
    if 買い方データ.正解 == "NG" and 判定結果 == "OK": # 誤りタイプ2
        # BCDを1ずつ増やし、Aを1減らす
        for 順序 in range(おやつの個数): # 順序は 0,1,2
            if 買い方データ.購入有無のリスト[順序]==1: # この部品がおやつを買う、と言った時
                おやつ担当部品のリスト[順序].増やす(1)
        部品A.減らす(1)
        
    print("正解は"+買い方データ.正解+"、その結果BCDAは ",end="")    
    print(",".join([str(x.中の値) for x in おやつ担当部品のリスト]),end="")
    print(","+str(部品A.中の値)) # Aの中の値も画面出力

## 【まずここを実行してみよう】

ここまで記述してきた内容に加え、関数を呼び出す側のメイン処理を加えたものをまとめて、以下に示します。下のプログラム中のどこかをクリックした状態で「シフトキー + Enter」を押すとプログラムが実行され、プログラムより下のところに出力結果が現れます。

In [5]:
class おやつの買い方(object):
    def __init__ (self,買い方の名前,購入有無のリスト,正解): # コンストラクタ
        self.買い方の名前=買い方の名前
        self.購入有無のリスト=購入有無のリスト # それぞれを「買うなら1、買わないなら0」
        self.正解=正解 # 「NGな買い方なら1、OKなら0」

class 部品(object):
    def __init__ (self,部品名,中の値): # コンストラクタ
        self.部品名=部品名
        self.中の値=中の値
    def 増やす(self,増やす値):   # 指定された値を足す
        self.中の値=self.中の値+増やす値
    def 減らす(self,減らす値):   # 指定された値を引く
        self.中の値=self.中の値-減らす値

おやつ担当部品のリスト=[部品("部品B",1), 部品("部品C",3), 部品("部品D",8)] # 3つの部品と中の値
部品A=部品("部品A",6)
おやつの個数=len(おやつ担当部品のリスト) # おやつの個数は、おやつ担当部品の数と同じ

教師データのリスト=[ # 5通りの教師データ
    おやつの買い方("買い方1",[1,1,1],"NG"),
    おやつの買い方("買い方2",[1,1,0],"NG"),
    おやつの買い方("買い方3",[1,0,1],"OK"),
    おやつの買い方("買い方4",[0,1,1],"OK"),
    おやつの買い方("買い方5",[1,0,0],"OK")]

未知データのリスト=[ # 3通りの未知データ
    おやつの買い方("買い方6",[0,1,0],"OK"),
    おやつの買い方("買い方7",[0,0,1],"OK"),
    おやつの買い方("買い方8",[0,0,0],"OK")]

def 買い方判定(買い方データ,部品A): # 部品B,C,D,Aに買い方の可否を判定させる
    合計=0
    for 順序 in range(おやつの個数):
        合計=合計+買い方データ.購入有無のリスト[順序]*おやつ担当部品のリスト[順序].中の値

    print(str(合計)+" > " + str(部品A.中の値)+" ", end="") # 比較の式を表示
    
    if 合計 > 部品A.中の値 : 
        return "NG" # 判定はNGで、部品Aが興奮する
    else:
        return "OK" # 判定はOK 

def 与えられたデータで学習(買い方データ,部品A):
    print(買い方データ.買い方の名前+"は ",end="")
    print(",".join([str(x) for x in 買い方データ.購入有無のリスト])+"、",end="")
    判定結果=買い方判定(買い方データ,部品A)
    print("の比較により判定は"+判定結果+"、",end="")

    # 判定結果と正解に基づき、BCDAの中の値を増減させる
    if 買い方データ.正解 == "OK" and 判定結果 == "NG": # 誤りタイプ1
        # BCDを1ずつ減らし、Aを1増やす
        for 順序 in range(おやつの個数): # 順序は 0,1,2
            if 買い方データ.購入有無のリスト[順序]==1: # この部品がおやつを買う、と言った時
                おやつ担当部品のリスト[順序].減らす(1)
        部品A.増やす(1)            
    if 買い方データ.正解 == "NG" and 判定結果 == "OK": # 誤りタイプ2
        # BCDを1ずつ増やし、Aを1減らす
        for 順序 in range(おやつの個数): # 順序は 0,1,2
            if 買い方データ.購入有無のリスト[順序]==1: # この部品がおやつを買う、と言った時
                おやつ担当部品のリスト[順序].増やす(1)
        部品A.減らす(1)
        
    print("正解は"+買い方データ.正解+"、その結果BCDAは ",end="")    
    print(",".join([str(x.中の値) for x in おやつ担当部品のリスト]),end="")
    print(","+str(部品A.中の値)) # Aの中の値も画面出力
################### 関数定義はここまでで終了 ###################

# メインの処理: とりあえず教師データ1個(買い方4)だけで判定させてみる
与えられたデータで学習(教師データのリスト[3],部品A)

買い方4は 0,1,1、11 > 6 の比較により判定はNG、正解はOK、その結果BCDAは 1,2,7,7


↑ BCDA は 1,2,7,7 になりましたね?

## 【次にここを実行しよう】

次に、買い方1から買い方5までを教師データとして、何度も繰り返し読み込んで学習させます。徐々に判定を誤らないようになっていきます。下のプログラム中のどこかをクリックした状態で「シフトキー + Enter」を押すとプログラムが実行され、プログラムより下のところに出力結果が現れます。

In [7]:
# 各部品の中の値を【まずここを実行してみよう】を行う前の状態に戻しておく
おやつ担当部品のリスト[0].中の値=1
おやつ担当部品のリスト[1].中の値=3
おやつ担当部品のリスト[2].中の値=8
部品A.中の値=6

# 繰り返し学習させる … 実行後にBCDAは4,5,2,7になるはず
for i in range(1,7): # 変数iは1から6まで
    for 教師データ in (教師データのリスト): # 5通りの教師データ
        与えられたデータで学習(教師データ,部品A)
    print("=== 学習の"+str(i)+"回目が終了 ===") 

[<__main__.部品 object at 0x7f06a81032b0>, <__main__.部品 object at 0x7f06a81032e8>, <__main__.部品 object at 0x7f06a8103320>]
買い方1は 1,1,1、12 > 6 の比較により判定はNG、正解はNG、その結果BCDAは 1,3,8,6
買い方2は 1,1,0、4 > 6 の比較により判定はOK、正解はNG、その結果BCDAは 2,4,8,5
買い方3は 1,0,1、10 > 5 の比較により判定はNG、正解はOK、その結果BCDAは 1,4,7,6
買い方4は 0,1,1、11 > 6 の比較により判定はNG、正解はOK、その結果BCDAは 1,3,6,7
買い方5は 1,0,0、1 > 7 の比較により判定はOK、正解はOK、その結果BCDAは 1,3,6,7
=== 学習の1回目が終了 ===
買い方1は 1,1,1、10 > 7 の比較により判定はNG、正解はNG、その結果BCDAは 1,3,6,7
買い方2は 1,1,0、4 > 7 の比較により判定はOK、正解はNG、その結果BCDAは 2,4,6,6
買い方3は 1,0,1、8 > 6 の比較により判定はNG、正解はOK、その結果BCDAは 1,4,5,7
買い方4は 0,1,1、9 > 7 の比較により判定はNG、正解はOK、その結果BCDAは 1,3,4,8
買い方5は 1,0,0、1 > 8 の比較により判定はOK、正解はOK、その結果BCDAは 1,3,4,8
=== 学習の2回目が終了 ===
買い方1は 1,1,1、8 > 8 の比較により判定はOK、正解はNG、その結果BCDAは 2,4,5,7
買い方2は 1,1,0、6 > 7 の比較により判定はOK、正解はNG、その結果BCDAは 3,5,5,6
買い方3は 1,0,1、8 > 6 の比較により判定はNG、正解はOK、その結果BCDAは 2,5,4,7
買い方4は 0,1,1、9 > 7 の比較により判定はNG、正解はOK、その結果BCDAは 2,4,3,8
買い方5は 1,0,0、2 > 8 の比較により判定はOK、正解はOK、その結果BCDAは 2,4,3,8
=== 学習の3回目が終了 ===
買い方1は 1,1,1

## 【任意課題】
<img src="http://pfe.p.cyber-u.ac.jp/img/Python/Week6/6-ex1.png" alt="任意課題(1)" style="width:360px">

<img src="http://pfe.p.cyber-u.ac.jp/img/Python/Week6/6-ex2.png" alt="任意課題(2)" style="width:360px">

<img src="http://pfe.p.cyber-u.ac.jp/img/Python/Week6/6-ex3.png" alt="任意課題(3)" style="width:360px">

In [ ]:
# 任意課題はこちらで↓
